#### smote + enn
          precision    recall  f1-score   support

          0       0.98      0.96      0.97      1920
          1       0.35      0.51      0.41        80

avg / total       0.95      0.94      0.95      2000

ACCURACY: 0.942

#### smote enn 결측치 2개 delete 했을 때

          precision    recall  f1-score   support

          0       0.98      0.95      0.96      1920
          1       0.28      0.51      0.37        80

avg / total       0.95      0.93      0.94      2000

ACCURACY: 0.929

#### ADASYN

          precision    recall  f1-score   support

          0       0.98      0.97      0.97      1920
          1       0.38      0.46      0.42        80

avg / total       0.95      0.95      0.95      2000

ACCURACY: 0.9485


#### Random oversampling
             precision    recall  f1-score   support

          0       0.98      0.94      0.96      1920
          1       0.30      0.65      0.41        80

avg / total       0.96      0.93      0.94      2000

ACCURACY: 0.9265


### 비식별처리 replace , fillna 해도 비식별처리로 인식됨

In [1]:
import pandas as pd
import numpy as np
import os
from imblearn.combine import SMOTEENN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import classification_report
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler ,normalize
from sklearn import preprocessing
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6


/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('./Data_set.csv',encoding='cp949')
df1 = pd.read_csv('./Test_set.csv',encoding='cp949')
df1.AGE.value_counts(0)

50    382
45    374
55    322
40    309
35    214
60    191
30     92
65     68
25     53
20      5
70      5
*       4
Name: AGE, dtype: int64

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100233 entries, 0 to 100232
Data columns (total 69 columns):
CUST_ID              100233 non-null int64
TARGET               100233 non-null int64
BNK_LNIF_CNT         100233 non-null int64
CPT_LNIF_CNT         100233 non-null int64
SPART_LNIF_CNT       100233 non-null int64
ECT_LNIF_CNT         100233 non-null int64
TOT_LNIF_AMT         100233 non-null int64
TOT_CLIF_AMT         100233 non-null int64
BNK_LNIF_AMT         100233 non-null int64
CPT_LNIF_AMT         100233 non-null int64
CRDT_OCCR_MDIF       100233 non-null int64
SPTCT_OCCR_MDIF      100233 non-null int64
CRDT_CARD_CNT        100233 non-null int64
CTCD_OCCR_MDIF       100233 non-null int64
CB_GUIF_CNT          100233 non-null int64
CB_GUIF_AMT          100233 non-null int64
OCCP_NAME_G          99769 non-null object
CUST_JOB_INCM        100233 non-null int64
HSHD_INFR_INCM       100233 non-null int64
ACTL_FMLY_NUM        100233 non-null int64
CUST_FMLY_NUM        100233 n

In [4]:
df.AGE

df.AGE = df.AGE.replace('*',46)
df.AGE = pd.Series(df.AGE).convert_objects(convert_numeric=True)
df.AGE = df.AGE.replace(0,df.AGE.mean())
df.AGE


/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


0         50
1         50
2         60
3         35
4         45
5         45
6         40
7         60
8         40
9         40
10        55
11        40
12        25
13        40
14        25
15        30
16        55
17        60
18        45
19        50
20        35
21        35
22        46
23        46
24        55
25        50
26        60
27        60
28        45
29        30
          ..
100203    50
100204    30
100205    35
100206    50
100207    60
100208    55
100209    45
100210    55
100211    50
100212    55
100213    60
100214    30
100215    45
100216    35
100217    35
100218    50
100219    55
100220    40
100221    50
100222    25
100223    40
100224    40
100225    25
100226    46
100227    55
100228    55
100229    45
100230    45
100231    45
100232    55
Name: AGE, Length: 100233, dtype: int64

In [5]:
df.SEX
df.SEX = df.SEX.replace('*',1)
df.SEX = pd.Series(df.SEX).convert_objects(convert_numeric=True)
df.SEX

/home/rosa/miniconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  This is separate from the ipykernel package so we can avoid doing imports until


0         1
1         1
2         2
3         1
4         1
5         1
6         2
7         1
8         2
9         1
10        2
11        1
12        2
13        1
14        2
15        1
16        2
17        2
18        2
19        2
20        1
21        2
22        1
23        1
24        1
25        2
26        2
27        1
28        2
29        1
         ..
100203    2
100204    2
100205    2
100206    2
100207    1
100208    1
100209    2
100210    2
100211    2
100212    2
100213    2
100214    1
100215    2
100216    1
100217    1
100218    2
100219    1
100220    2
100221    2
100222    1
100223    1
100224    1
100225    1
100226    1
100227    1
100228    2
100229    2
100230    1
100231    1
100232    2
Name: SEX, Length: 100233, dtype: int64

In [6]:
df.TEL_CNTT_QTR = df.TEL_CNTT_QTR.astype(str)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100233 entries, 0 to 100232
Data columns (total 69 columns):
CUST_ID              100233 non-null int64
TARGET               100233 non-null int64
BNK_LNIF_CNT         100233 non-null int64
CPT_LNIF_CNT         100233 non-null int64
SPART_LNIF_CNT       100233 non-null int64
ECT_LNIF_CNT         100233 non-null int64
TOT_LNIF_AMT         100233 non-null int64
TOT_CLIF_AMT         100233 non-null int64
BNK_LNIF_AMT         100233 non-null int64
CPT_LNIF_AMT         100233 non-null int64
CRDT_OCCR_MDIF       100233 non-null int64
SPTCT_OCCR_MDIF      100233 non-null int64
CRDT_CARD_CNT        100233 non-null int64
CTCD_OCCR_MDIF       100233 non-null int64
CB_GUIF_CNT          100233 non-null int64
CB_GUIF_AMT          100233 non-null int64
OCCP_NAME_G          99769 non-null object
CUST_JOB_INCM        100233 non-null int64
HSHD_INFR_INCM       100233 non-null int64
ACTL_FMLY_NUM        100233 non-null int64
CUST_FMLY_NUM        100233 n

In [8]:
# df.PAYM_METD = df.PAYM_METD.fillna('O')
# df.TEL_MBSP_GRAD = df.TEL_MBSP_GRAD.fillna('R')
# df.MATE_OCCP_NAME_G = df.MATE_OCCP_NAME_G.fillna('주부')
# df.OCCP_NAME_G = df.OCCP_NAME_G.fillna('주부')
# df.LAST_CHLD_AGE = df.LAST_CHLD_AGE.fillna(df.LAST_CHLD_AGE.mean())

In [9]:
# # *가 있으면 그 줄을 그냥 지워버려라
# for i in range(100234):
#     print (df['OCCCP_NAME_G'](i) is '*')
        
    
    

In [10]:
#무조건
del df['CUST_ID']
#null
del df['MATE_OCCP_NAME_G']
del df['OCCP_NAME_G']
del df['LAST_CHLD_AGE']
del df['TEL_MBSP_GRAD']
del df['PAYM_METD']


In [11]:
#값이 오브젝트인거 다뺏을때
del df['LINE_STUS']
del df['CBPT_MBSP_YN']

In [12]:
df = pd.get_dummies(df)

In [13]:
df_test = df[:2000]
df_train = df[2000:]

In [14]:
x_data = df_train.drop('TARGET',axis=1)
y_data = df_train.TARGET

In [15]:
X = x_data
y = y_data
from imblearn.over_sampling import ADASYN 
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import CondensedNearestNeighbour 

# undersampling
# cnn = CondensedNearestNeighbour(random_state=42) 
# X_resampled, y_resampled = cnn.fit_sample(X, y)

#RANDOM 
# ros = RandomOverSampler(random_state=42)
# X_resampled, y_resampled = ros.fit_sample(X, y)

#ADASYN
# ada = ADASYN(random_state=42,n_neighbors=3)
# X_resampled, y_resampled = ada.fit_sample(X, y)

# Apply SMOTE + ENN
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_sample(X, y)

In [16]:
df_1 = pd.DataFrame(X_resampled)
df_2 = pd.DataFrame(y_resampled)
df_2.columns = ['TARGET']
df_final = pd.concat([df_1,df_2],axis=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,149,150,151,152,153,154,155,156,157,TARGET
0,0.0,0.0,1.0,1.0,48001.0,48001.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2.0,0.0,0.0,0.0,48001.0,1.0,48001.0,0.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.0,0.0,0.0,0.0,9001.0,9001.0,9001.0,0.0,25.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2.0,0.0,0.0,0.0,39001.0,39001.0,39001.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,1.0,132001.0,132001.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [17]:
df_final = df_final.sample(frac=1)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,149,150,151,152,153,154,155,156,157,TARGET
115300,1.000000,2.000000,2.746354,0.746354,31120.531269,31120.531269,6761.937462,18001.000000,16.043750,60.825001,...,0.0,0.373177,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
87876,0.071863,0.143726,1.071863,0.928137,4078.946781,3216.589356,431.250576,646.839932,19.898859,12.065779,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
121896,0.000000,2.977202,5.908809,2.931607,32248.673096,32248.673096,0.000000,2932.606645,2.094294,13.820720,...,0.0,0.977202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
50981,1.000000,0.000000,1.000000,1.000000,72001.000000,48001.000000,48001.000000,0.000000,25.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
122179,0.000000,2.010814,2.010814,0.000000,7044.254909,7044.254909,0.000000,7044.254909,8.956745,25.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [18]:
train_y = df_final.TARGET
train_x = df_final.drop('TARGET',axis=1)

In [19]:
test_y = df_test.TARGET
test_x = df_test.drop('TARGET',axis=1)

In [20]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0,5))
train_x = min_max_scaler.fit_transform(train_x)
test_x = min_max_scaler.fit_transform(test_x)

In [21]:
from sklearn import linear_model
reg = linear_model.LogisticRegression()
reg.fit(train_x,train_y)
score = reg.score(test_x,test_y)
print('Logistic Regression Accuracy : ',score)

Logistic Regression Accuracy :  0.8265


In [22]:
from sklearn.metrics import classification_report
print(classification_report(test_y,reg.predict(test_x)))

             precision    recall  f1-score   support

          0       0.99      0.83      0.90      1920
          1       0.16      0.81      0.27        80

avg / total       0.96      0.83      0.88      2000



In [23]:
reg2 = linear_model.LogisticRegression(class_weight='balanced')
reg2.fit(train_x,train_y)
score2 = reg2.score(test_x,test_y)
print('Logistic Regression used class weight ' ,score2)
print(classification_report(test_y,reg2.predict(test_x)))

Logistic Regression used class weight  0.8445
             precision    recall  f1-score   support

          0       0.99      0.85      0.91      1920
          1       0.17      0.76      0.28        80

avg / total       0.96      0.84      0.89      2000



In [24]:
# fit model no training data
model = XGBClassifier(learning_rate=0.2,max_depth=7,base_score=0.5,max_delta_step=0,n_estimators=100)

pipe = Pipeline([['xg',MinMaxScaler(feature_range=(0,5))],['model',model]])
pipe.fit(train_x,train_y)
print(classification_report(test_y,pipe.predict(test_x)))
print("ACCURACY:",pipe.score(test_x,test_y))

             precision    recall  f1-score   support

          0       0.98      0.94      0.96      1920
          1       0.27      0.53      0.36        80

avg / total       0.95      0.92      0.94      2000

ACCURACY: 0.9245
